In [38]:
import os
import random

import numpy as np # linear algebra
import shutil
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# from PIL import Image
from torchvision.io import read_image
from matplotlib import pyplot as plt

torch.manual_seed(50)

# Getting the dataset from kaggle into the colab notebook and extracting it.

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

In [ ]:
#Make directory name kaggle
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#Copy the json kaggle to this directory
! cp kaggle.json ~/.kaggle/

In [ ]:
#Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# downloading the dataset
! kaggle datasets download -d tawsifurrahman/covid19-radiography-database

 98% 761M/778M [00:13<00:00, 85.9MB/s]
100% 778M/778M [00:14<00:00, 58.2MB/s]


In [ ]:
!sudo rm -r ./COVID-19_Radiography_Dataset

rm: cannot remove './COVID-19_Radiography_Dataset': No such file or directory


In [39]:
#extracting the zip files from the dataset

# Put on the same directory
from zipfile import ZipFile 
  
# specifying the name of the zip file
file = "covid19-radiography-database.zip"
  
# open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # list all the contents of the zip file
    zip.printdir() 
  
    # extract all files
    print('extraction...') 
    zip.extractall() 
    print('Done!')

Streaming output truncated to the last 5000 lines.
COVID-19_Radiography_Dataset/Normal/masks/Normal-7923.png 2022-03-19 13:41:48         1507
COVID-19_Radiography_Dataset/Normal/masks/Normal-7924.png 2022-03-19 13:41:48         1453
COVID-19_Radiography_Dataset/Normal/masks/Normal-7925.png 2022-03-19 13:41:48         1435
COVID-19_Radiography_Dataset/Normal/masks/Normal-7926.png 2022-03-19 13:41:48         1663
COVID-19_Radiography_Dataset/Normal/masks/Normal-7927.png 2022-03-19 13:41:48         1374
COVID-19_Radiography_Dataset/Normal/masks/Normal-7928.png 2022-03-19 13:41:48         1522
COVID-19_Radiography_Dataset/Normal/masks/Normal-7929.png 2022-03-19 13:41:48         1350
COVID-19_Radiography_Dataset/Normal/masks/Normal-793.png 2022-03-19 13:41:48         1516
COVID-19_Radiography_Dataset/Normal/masks/Normal-7930.png 2022-03-19 13:41:48         1607
COVID-19_Radiography_Dataset/Normal/masks/Normal-7931.png 2022-03-19 13:41:48         1536
COVID-19_Radiography_Dataset/Normal/mask

# Keeping the required files and formatting the folders as required by pytorch.

In [40]:
!sudo rm -r ./COVID-19_Radiography_Dataset/COVID/masks
!sudo rm -r ./COVID-19_Radiography_Dataset/Normal/masks
!sudo rm -r ./COVID-19_Radiography_Dataset/Viral\ Pneumonia/masks

In [41]:
for i in os.listdir("/content/COVID-19_Radiography_Dataset/COVID/images"):
  source_path = os.path.join("/content/COVID-19_Radiography_Dataset/COVID/images", i)
  target_path = "/content/COVID-19_Radiography_Dataset/COVID/"
  shutil.move(source_path, target_path)

In [42]:
!sudo rm -r ./COVID-19_Radiography_Dataset/COVID/images

In [43]:
for i in os.listdir("/content/COVID-19_Radiography_Dataset/Normal/images"):
  source_path = os.path.join("/content/COVID-19_Radiography_Dataset/Normal/images", i)
  target_path = "/content/COVID-19_Radiography_Dataset/Normal/"
  shutil.move(source_path, target_path)

In [44]:
!sudo rm -r ./COVID-19_Radiography_Dataset/Normal/images

In [45]:
for i in os.listdir("/content/COVID-19_Radiography_Dataset/Viral Pneumonia/images"):
  source_path = os.path.join("/content/COVID-19_Radiography_Dataset/Viral Pneumonia/images", i)
  target_path = "/content/COVID-19_Radiography_Dataset/Viral Pneumonia/"
  shutil.move(source_path, target_path)

In [46]:
!sudo rm -r ./COVID-19_Radiography_Dataset/Viral\ Pneumonia/images

In [47]:
!sudo rm -r ./COVID-19_Radiography_Dataset/Lung_Opacity/

In [48]:
!sudo rm ./COVID-19_Radiography_Dataset/Lung_Opacity.metadata.xlsx

Setting up the classes and source directories as per the dataset received.

In [50]:
class_names = ['normal', 'viral', 'covid']
root_dir = 'COVID-19_Radiography_Dataset'
source_dirs = ['Normal', 'Viral Pneumonia', 'COVID']

if os.path.isdir(os.path.join(root_dir, source_dirs[1])):
    os.mkdir(os.path.join(root_dir, 'test'))

    for i, d in enumerate(source_dirs):
        os.rename(os.path.join(root_dir, d), os.path.join(root_dir, class_names[i]))

    for c in class_names:
        os.mkdir(os.path.join(root_dir, 'test', c))

    for c in class_names:
        images = [x for x in os.listdir(os.path.join(root_dir, c)) if x.lower().endswith('png')]
        selected_images = random.sample(images, 30)
        for image in selected_images:
            source_path = os.path.join(root_dir, c, image)
            target_path = os.path.join(root_dir, 'test', c, image)
            shutil.move(source_path, target_path)

FileExistsError: ignored

Custom dataset class as required by pytorch.

In [51]:
class ChestXRayDataset(torch.utils.data.Dataset):
    def __init__(self, image_dirs, transform):
        def get_images(class_name):
            images = [x for x in os.listdir(image_dirs[class_name]) if x.lower().endswith('png')]
            print(f'Found {len(images)} {class_name} examples')
            return images
        
        self.images = {}
        self.class_names = ['normal', 'viral', 'covid']
        
        for class_name in self.class_names:
            self.images[class_name] = get_images(class_name)
            
        self.image_dirs = image_dirs
        self.transform = transform
        
    
    def __len__(self):
        return sum([len(self.images[class_name]) for class_name in self.class_names])
    
    
    def __getitem__(self, index):
        class_name = random.choice(self.class_names)
        index = index % len(self.images[class_name])
        image_name = self.images[class_name][index]
        image_path = os.path.join(self.image_dirs[class_name], image_name)
        image = Image.open(image_path) #.convert('RGB')
        return self.transform(image), self.class_names.index(class_name)

# Simple single transform applied.

In [ ]:
# simple transform applied of only ToTensor.
simple_transforms_train = transforms.Compose([transforms.ToTensor(),])

In [52]:
# simple transform applied to resize

train_resize = transforms.Compose(transforms.Resize((224, 224)))

In [54]:
# simple transform applied for Histogram Equalization

train_histgram = transforms.Compose(transforms.functional.equalize(())

TypeError: ignored

In [ ]:
# simple transform applied for Gaussian Blur

train_blurrer = transforms.Compose(transforms.GaussianBlur(kernel_size=(5,9), sigma= (0.1, 5)))

In [ ]:
train_dirs = {
    'normal': f'{root_dir}/normal',
    'viral': f'{root_dir}/viral',
    'covid': f'{root_dir}/covid'
}

train_dataset = ChestXRayDataset(train_dirs, simple_transforms_train)

Found 10162 normal examples
Found 1315 viral examples
Found 3586 covid examples


In [ ]:
simple_transforms_test = transforms.Compose([
                                       transforms.ToTensor(),
                                       ])

In [ ]:
test_dirs = {
    'normal': f'{root_dir}/test/normal',
    'viral': f'{root_dir}/test/viral',
    'covid': f'{root_dir}/test/covid'
}

test_dataset = ChestXRayDataset(test_dirs, simple_transforms_test)

Found 30 normal examples
Found 30 viral examples
Found 30 covid examples


# Multiple Transforms applied.

In [ ]:
multiple_transforms_train = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       torchvision.transforms.Normalize((0.5,), (0.5,)),
                                       torchvision.transforms.GaussianBlur(11, sigma=(0.1, 2.0))
                                       ])

In [ ]:
train_dirs = {
    'normal': f'{root_dir}/normal',
    'viral': f'{root_dir}/viral',
    'covid': f'{root_dir}/covid'
}

train_dataset = ChestXRayDataset(train_dirs, multiple_transforms_train)

Found 10162 normal examples
Found 1315 viral examples
Found 3586 covid examples


In [ ]:
multiple_transforms_test = transforms.Compose([
                                       transforms.ToTensor(),
                                       torchvision.transforms.Normalize((0.5,), (0.5,))
                                       ])
# observe that you only need to make these transformations to the test set 
# because in testing environment our model will get normal images and we don't impose specific conditions 
# on the images while testing.

In [ ]:
test_dirs = {
    'normal': f'{root_dir}/test/normal',
    'viral': f'{root_dir}/test/viral',
    'covid': f'{root_dir}/test/covid'
}

test_dataset = ChestXRayDataset(test_dirs, multiple_transforms_test)

Found 30 normal examples
Found 30 viral examples
Found 30 covid examples


# Segregating the model in batches

In [ ]:
batch_size = 6

dl_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dl_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print('Number of training batches', len(dl_train))
print('Number of test batches', len(dl_test))

Number of training batches 2511
Number of test batches 15


# Visualising images

In [ ]:
train_dataset.class_names

['normal', 'viral', 'covid']

In [ ]:
class_names = train_dataset.class_names

def show_images(images, labels, preds):
    plt.figure(figsize=(8, 4))
    for i, image in enumerate(images):
        plt.subplot(1, 6, i + 1, xticks=[], yticks=[])
        image = image.numpy().transpose((1, 2, 0))
        mean = np.array([0.5]) #torchvision.transforms.Normalize((0.5,), (0.5,))
        std = np.array([0.5])
        image = image * std + mean
        image = np.clip(image, 0., 1.)
        if image.shape[2] == 1:
          image = image[:, :, 0]
        plt.imshow(image)
        col = 'green'
        if preds[i] != labels[i]:
            col = 'red'
            
        plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
        plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=col)
    plt.tight_layout()
    plt.show()


In [ ]:
images, labels = next(iter(dl_train))
show_images(images, labels, labels)

NameError: ignored